In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-12-01 14:32:56.486560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 14:32:56.590749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 14:32:56.590766: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-01 14:32:56.612317: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 14:32:57.142373: W tensorflow/stream_executor/platform/de

In [2]:
list_f=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
name_all = []
auc_roc_test=[] 
for feature in list_f:
    #read main set
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_xgb_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_xgb_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('xgb_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('xgb_'+feature+'.csv')

    #read external set
    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_xgb_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_xgb_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_xgb_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_xgb_'+feature+'.csv')

    #read pred set and get the Y(pred set) for predicting X(pred set)
    read_p=pd.read_csv('pred_set_plus_'+feature+'.csv')
    df_p=pd.DataFrame(read_p)
    X_p=df_p.iloc[:,2:]
     
    #get X and Y (main set and external set)
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    
    #algorithm
    xgboost_model = xgboost.XGBClassifier(
      max_depth=5,
      learning_rate=0.05,
      n_estimators=3000,
      gamma=0.0,
      min_child_weight=5,
      max_delta_step=1,
      subsample=0.53,
      colsample_bytree=0.66,
      colsample_bylevel=1,
      reg_alpha=0,
      reg_lambda=1,
      scale_pos_weight=1,
      base_score=0.5,
      seed=2016)
    
    #train algorithm
    xgboost_model.fit(X, y)

    
    #predict the X of main set
    proba = xgboost_model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_xgb_main_set_'+feature+'.csv',index=False)
 
    #predict the X of pred set
    proba = xgboost_model.predict_proba(X_p)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_pred=pd.DataFrame()
    df_pred['pred']=pro
    df_pred.to_csv('pred_xgb_pred_set_'+feature+'.csv',index=False)

    #predict the X of external set
    proba = xgboost_model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_xgb_external_set_'+feature+'.csv',index=False)
    
    #calculate the AUC-ROC of the baseline models
    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    name = 'xgb_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    xgboost_model=''

In [3]:
for feature in list_f:
    #read main set
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    a = df['active']
    c=[]
    for b in a:
        if b==1:
            b=0
        else:
            b=1
        c.append(b)
    df.insert(loc=2, column='inactive', value=c)
    cl = df.columns.tolist()
    feat = cl[3:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active','inactive']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    lab_ta_fea.remove('inactive')
    df.to_csv("main_set_plus_"+feature+"_fcnn_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_fcnn_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('fcnn_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active","inactive"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('fcnn_'+feature+'.csv')

    #read external set
    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    a1 = df_e['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)
    df_e.insert(loc=2, column='inactive', value=c1)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[3:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active','inactive']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    lab_ta_fea_e.remove('inactive')
    df_e.to_csv("external_set_plus_"+feature+"_fcnn_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_fcnn_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_fcnn_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active","inactive"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_fcnn_'+feature+'.csv')
    
    #read pred set and get the Y(pred set) for predicting X(pred set)
    read_p=pd.read_csv('pred_set_plus_'+feature+'.csv')
    df_p=pd.DataFrame(read_p)
    X_p=df_p.iloc[:,2:]

    #get X and Y (main set and external set)
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w
    
    #algorithm
    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(feat),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    
    #train algorithm
    model.fit(X, y,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    
    #predict the X of main set
    proba=model.predict(X,
                      batch_size=None,
                      verbose="auto",
                      steps=None,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False)
    pro=[]
    for p in proba:
        pi = p[0]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_fcnn_main_set_'+feature+'.csv',index=False)

    #predict the X of pred set
    proba=model.predict(X_p,
                      batch_size=None,
                      verbose="auto",
                      steps=None,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False)
    pro=[]
    for p in proba:
        pi = p[0]
        pro.append(pi)
    df_pred=pd.DataFrame()
    df_pred['pred']=pro
    df_pred.to_csv('pred_fcnn_pred_set_'+feature+'.csv',index=False)
    
    #predict the X of external set   
    proba=model.predict(X_e,
                      batch_size=None,
                      verbose="auto",
                      steps=None,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False)
    
    #calculate the AUC-ROC of the baseline models
    pro=[]
    for p in proba:
        pi = p[0]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_fcnn_external_set_'+feature+'.csv',index=False)
    name = 'fcnn_'+feature
    name_all.append(name)
    globals()[name] = score[1]
    auc_roc_test.append(globals()[name])
    model=''

Epoch 1/100


2022-12-01 14:39:56.298803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 14:39:56.299399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-12-01 14:39:56.299450: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-12-01 14:39:56.299490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-12-01 14:39:56.299528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 1s 503ms/step - loss: 66.7844 - auc: 0.4610
Epoch 2/100
1/1 [==============================] - 0s 98ms/step - loss: 63.1592 - auc: 0.7993
Epoch 3/100
1/1 [==============================] - 0s 97ms/step - loss: 59.1677 - auc: 0.7912
Epoch 4/100
1/1 [==============================] - 0s 104ms/step - loss: 54.8231 - auc: 0.8000
Epoch 5/100
1/1 [==============================] - 0s 104ms/step - loss: 50.3775 - auc: 0.8406
Epoch 6/100
1/1 [==============================] - 0s 102ms/step - loss: 46.1334 - auc: 0.8721
Epoch 7/100
1/1 [==============================] - 0s 91ms/step - loss: 42.7456 - auc: 0.7631
Epoch 8/100
1/1 [==============================] - 0s 85ms/step - loss: 40.1048 - auc: 0.5360
Epoch 9/100
1/1 [==============================] - 0s 84ms/step - loss: 36.7454 - auc: 0.7237
Epoch 10/100
1/1 [==============================] - 0s 87ms/step - loss: 33.6951 - auc: 0.8865
Epoch 11/100
1/1 [==============================] - 0s 85ms/step - 

Epoch 88/100
1/1 [==============================] - 0s 95ms/step - loss: 0.5433 - auc: 0.9195
Epoch 89/100
1/1 [==============================] - 0s 98ms/step - loss: 0.5482 - auc: 0.9141
Epoch 90/100
1/1 [==============================] - 0s 97ms/step - loss: 0.5294 - auc: 0.9192
Epoch 91/100
1/1 [==============================] - 0s 99ms/step - loss: 0.5345 - auc: 0.9161
Epoch 92/100
1/1 [==============================] - 0s 98ms/step - loss: 0.5243 - auc: 0.9189
Epoch 93/100
1/1 [==============================] - 0s 94ms/step - loss: 0.5284 - auc: 0.9162
Epoch 94/100
1/1 [==============================] - 0s 91ms/step - loss: 0.5095 - auc: 0.9212
Epoch 95/100
1/1 [==============================] - 0s 97ms/step - loss: 0.5102 - auc: 0.9196
Epoch 96/100
1/1 [==============================] - 0s 93ms/step - loss: 0.4977 - auc: 0.9227
Epoch 97/100
1/1 [==============================] - 0s 95ms/step - loss: 0.4987 - auc: 0.9209
Epoch 98/100
1/1 [==============================] - 0s 94ms/

1/1 [==============================] - 0s 24ms/step - loss: 0.6802 - auc: 0.7699
Epoch 74/100
1/1 [==============================] - 0s 20ms/step - loss: 0.6734 - auc: 0.7707
Epoch 75/100
1/1 [==============================] - 0s 22ms/step - loss: 0.6835 - auc: 0.7675
Epoch 76/100
1/1 [==============================] - 0s 19ms/step - loss: 0.6714 - auc: 0.7697
Epoch 77/100
1/1 [==============================] - 0s 21ms/step - loss: 0.6439 - auc: 0.7847
Epoch 78/100
1/1 [==============================] - 0s 21ms/step - loss: 0.6529 - auc: 0.7792
Epoch 79/100
1/1 [==============================] - 0s 20ms/step - loss: 0.6717 - auc: 0.7706
Epoch 80/100
1/1 [==============================] - 0s 20ms/step - loss: 0.6836 - auc: 0.7671
Epoch 81/100
1/1 [==============================] - 0s 21ms/step - loss: 0.6685 - auc: 0.7702
Epoch 82/100
1/1 [==============================] - 0s 19ms/step - loss: 0.6464 - auc: 0.7796
Epoch 83/100
1/1 [==============================] - 0s 21ms/step - loss: 

1/1 [==============================] - 0s 97ms/step - loss: 1.1957 - auc: 0.9569
Epoch 59/100
1/1 [==============================] - 0s 96ms/step - loss: 1.1472 - auc: 0.9527
Epoch 60/100
1/1 [==============================] - 0s 101ms/step - loss: 1.0721 - auc: 0.9570
Epoch 61/100
1/1 [==============================] - 0s 98ms/step - loss: 1.0363 - auc: 0.9503
Epoch 62/100
1/1 [==============================] - 0s 103ms/step - loss: 0.9753 - auc: 0.9545
Epoch 63/100
1/1 [==============================] - 0s 103ms/step - loss: 0.9288 - auc: 0.9543
Epoch 64/100
1/1 [==============================] - 0s 99ms/step - loss: 0.8902 - auc: 0.9539
Epoch 65/100
1/1 [==============================] - 0s 103ms/step - loss: 0.8403 - auc: 0.9562
Epoch 66/100
1/1 [==============================] - 0s 102ms/step - loss: 0.8148 - auc: 0.9528
Epoch 67/100
1/1 [==============================] - 0s 98ms/step - loss: 0.7624 - auc: 0.9597
Epoch 68/100
1/1 [==============================] - 0s 96ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 1.7110 - auc: 0.8462
Epoch 44/100
1/1 [==============================] - 0s 25ms/step - loss: 1.7380 - auc: 0.8480
Epoch 45/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6636 - auc: 0.8564
Epoch 46/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6407 - auc: 0.8630
Epoch 47/100
1/1 [==============================] - 0s 22ms/step - loss: 1.6312 - auc: 0.8590
Epoch 48/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5438 - auc: 0.8664
Epoch 49/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5962 - auc: 0.8517
Epoch 50/100
1/1 [==============================] - 0s 23ms/step - loss: 1.5572 - auc: 0.8598
Epoch 51/100
1/1 [==============================] - 0s 24ms/step - loss: 1.5350 - auc: 0.8622
Epoch 52/100
1/1 [==============================] - 0s 23ms/step - loss: 1.4708 - auc: 0.8746
Epoch 53/100
1/1 [==============================] - 0s 25ms/step - loss: 

1/1 [==============================] - 0s 99ms/step - loss: 6.6724 - auc: 0.8572
Epoch 29/100
1/1 [==============================] - 0s 93ms/step - loss: 6.1559 - auc: 0.8452
Epoch 30/100
1/1 [==============================] - 0s 96ms/step - loss: 5.6784 - auc: 0.8445
Epoch 31/100
1/1 [==============================] - 0s 96ms/step - loss: 5.2310 - auc: 0.8465
Epoch 32/100
1/1 [==============================] - 0s 92ms/step - loss: 4.8062 - auc: 0.8617
Epoch 33/100
1/1 [==============================] - 0s 95ms/step - loss: 4.4449 - auc: 0.8664
Epoch 34/100
1/1 [==============================] - 0s 98ms/step - loss: 4.1526 - auc: 0.8559
Epoch 35/100
1/1 [==============================] - 0s 97ms/step - loss: 3.8438 - auc: 0.8560
Epoch 36/100
1/1 [==============================] - 0s 100ms/step - loss: 3.5528 - auc: 0.8592
Epoch 37/100
1/1 [==============================] - 0s 96ms/step - loss: 3.2936 - auc: 0.8552
Epoch 38/100
1/1 [==============================] - 0s 93ms/step - loss:

1/1 [==============================] - 0s 20ms/step - loss: 1.6708 - auc: 0.5398
Epoch 14/100
1/1 [==============================] - 0s 20ms/step - loss: 1.4501 - auc: 0.6595
Epoch 15/100
1/1 [==============================] - 0s 20ms/step - loss: 1.3255 - auc: 0.7548
Epoch 16/100
1/1 [==============================] - 0s 18ms/step - loss: 1.3171 - auc: 0.7970
Epoch 17/100
1/1 [==============================] - 0s 20ms/step - loss: 1.3974 - auc: 0.8047
Epoch 18/100
1/1 [==============================] - 0s 20ms/step - loss: 1.4522 - auc: 0.8066
Epoch 19/100
1/1 [==============================] - 0s 19ms/step - loss: 1.4117 - auc: 0.8108
Epoch 20/100
1/1 [==============================] - 0s 20ms/step - loss: 1.3783 - auc: 0.8079
Epoch 21/100
1/1 [==============================] - 0s 18ms/step - loss: 1.2610 - auc: 0.8126
Epoch 22/100
1/1 [==============================] - 0s 21ms/step - loss: 1.2422 - auc: 0.7897
Epoch 23/100
1/1 [==============================] - 0s 20ms/step - loss: 

14/14 [==============================] - 0s 710us/step


In [4]:
for feature in list_f:
    #read main set
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    feat = cl[2:]
    leng = len(feat)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    lab_ta_fea.remove('smiles')
    lab_ta_fea.remove('active')
    df.to_csv("main_set_plus_"+feature+"_lr_columns.csv",index=False)
    read_c = pd.read_csv("main_set_plus_"+feature+"_lr_columns.csv")
    df_main = pd.DataFrame(read_c)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_main.to_csv('lr_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset = loader.create_dataset('lr_'+feature+'.csv')

    #read external set
    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    feat_e = cl_e[2:]
    leng_e = len(feat_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    lab_ta_fea_e.remove('smiles')
    lab_ta_fea_e.remove('active')
    df_e.to_csv("external_set_plus_"+feature+"_lr_columns.csv",index=False)
    read_c_e = pd.read_csv("external_set_plus_"+feature+"_lr_columns.csv")
    df_external = pd.DataFrame(read_c_e)
    featurizer = dc.feat.UserDefinedFeaturizer(lab_ta_fea_e)
    with dc.utils.UniversalNamedTemporaryFile(mode='w') as tmpfile:

      df_external.to_csv('external_lr_'+feature+'.csv')

      loader = dc.data.UserCSVLoader(["active"], id_field="smiles",

                                 featurizer=featurizer)

      dataset_e = loader.create_dataset('external_lr_'+feature+'.csv')

    #read pred set and get the Y(pred set) for predicting X(pred set)
    read_p=pd.read_csv('pred_set_plus_'+feature+'.csv')
    df_p=pd.DataFrame(read_p)
    X_p=df_p.iloc[:,2:]
    
    #get X and Y (main set and external set)
    X, y, w = dataset.X, dataset.y, dataset.w
    X_e, y_e, w_e = dataset_e.X, dataset_e.y, dataset_e.w

    #algorithm
    sklearn_model = LogisticRegression(
          penalty= 'l2',
          C=1.0,
          class_weight="balanced",
          dual=False,
          fit_intercept=True,
          max_iter=100,
          multi_class='ovr',
          n_jobs=-1,
          random_state=None,solver='liblinear',tol=0.0001,verbose=0,warm_start=False)

    #train algorithm
    sklearn_model.fit(X, y)
    
    #predict the X of main set 
    proba = sklearn_model.predict_proba(X)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_main=pd.DataFrame()
    df_main['pred']=pro
    df_main.to_csv('pred_lr_main_set_'+feature+'.csv',index=False)

    #predict the X of pred set 
    proba = sklearn_model.predict_proba(X_p)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_pred=pd.DataFrame()
    df_pred['pred']=pro
    df_pred.to_csv('pred_lr_pred_set_'+feature+'.csv',index=False)
        
    #predict the X of external set 
    proba = sklearn_model.predict_proba(X_e)
    pro=[]
    for p in proba:
        pi = p[1]
        pro.append(pi)
    df_external=pd.DataFrame()
    df_external['pred']=pro
    df_external.to_csv('pred_lr_external_set_'+feature+'.csv',index=False)
    
    #calculate the AUC-ROC of the baseline models
    y_l=[]
    for y in y_e:
        y = int(y)
        y_l.append(y)
    auc_roc=pd.DataFrame()
    auc_roc['label'] = y_l
    auc_roc['pred'] = pro
    y_label = auc_roc['label']
    y_pred = auc_roc['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    roc_auc = auc(fpr, tpr)
    name = 'lr_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])
    sklearn_model=''

/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [5]:
for feature in list_f:
    #read main set
    read = pd.read_csv("main_set_plus_"+feature+".csv")
    df = pd.DataFrame(read)
    cl = df.columns.tolist()
    fea = cl[2:]
    leng = len(fea)
    lab_ta_fea = ['smiles','active']
    for nu in range(1,leng+1):
        nu = 'desc'+str(nu)
        lab_ta_fea.append(nu)
    df.columns = lab_ta_fea
    df2=pd.DataFrame()
    df2['smiles']=df['smiles']
    df2['active']=df['active']
    df2.to_csv('main_set_plus_'+feature+'_dmpnn_smile_active.csv',index=False)
    df.pop('smiles')
    df.pop('active')
    df.to_csv('main_set_plus_'+feature+'_dmpnn_feature.csv',index=False)

    #read external set
    read_e = pd.read_csv("external_set_plus_"+feature+".csv")
    df_e = pd.DataFrame(read_e)
    cl_e = df_e.columns.tolist()
    fea_e = cl_e[2:]
    leng_e = len(fea_e)
    lab_ta_fea_e = ['smiles','active']
    for nu_e in range(1,leng_e+1):
        nu_e = 'desc'+str(nu_e)
        lab_ta_fea_e.append(nu_e)
    df_e.columns = lab_ta_fea_e
    df2_e=pd.DataFrame()
    df2_e['smiles']=df_e['smiles']
    df2_e['active']=df_e['active']
    label=df_e['active']
    df2_e.to_csv('external_set_plus_'+feature+'_dmpnn_smile_active.csv',index=False)
    df_e.pop('smiles')
    df_e.pop('active')
    df_e.to_csv('external_set_plus_'+feature+'_feature.csv',index=False)
    
    #train D-MPNN
    arguments = [
        '--data_path', 'main_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_only',
        '--features_path','main_set_plus_'+feature+'_dmpnn_feature.csv',
        '--dataset_type', 'classification',
        '--save_dir', 'dmpnn_'+feature+'_checkpoints',
    ]

    args = chemprop.args.TrainArgs().parse_args(arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    #predict the X of main set
    arguments = [
        '--test_path', 'main_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_path','main_set_plus_'+feature+'_dmpnn_feature.csv',
        '--preds_path', 'dmpnn_'+feature+'_pred_main.csv',
        '--checkpoint_dir', 'dmpnn_'+feature+'_checkpoints'
    ]
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)
    file = pd.read_csv('dmpnn_'+feature+'_pred_main.csv')
    pred_df = pd.DataFrame(file)
    pred=pd.DataFrame()
    pred['pred'] =pred_df['active']
    df_main=pd.DataFrame()
    df_main['pred']=pred
    df_main.to_csv('pred_dmpnn_main_set_'+feature+'.csv',index=False)
        
    #predict the X of pred set
    read_p=pd.read_csv('pred_set_plus_'+feature+'.csv')
    df_p=pd.DataFrame(read_p)
    X_p=df_p.iloc[:,2:]
    X_p.to_csv('pred_set_plus_'+feature+'_feature.csv',index=False)
    X_s=df_p.loc[:,['smiles']]
    X_s.to_csv('pred_set_plus_'+feature+'_dmpnn_smile.csv',index=False)
    arguments = [
        '--test_path', 'pred_set_plus_'+feature+'_dmpnn_smile.csv',
        '--features_path','pred_set_plus_'+feature+'_feature.csv',
        '--preds_path', 'dmpnn_'+feature+'_pred_pred.csv',
        '--checkpoint_dir', 'dmpnn_'+feature+'_checkpoints'
    ]
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)
    file = pd.read_csv('dmpnn_'+feature+'_pred_pred.csv')
    pred_df = pd.DataFrame(file)
    pred=pd.DataFrame()
    pred['pred'] =pred_df['active']
    df_pred=pd.DataFrame()
    df_pred['pred']=pred
    df_pred.to_csv('pred_dmpnn_pred_set_'+feature+'.csv',index=False)
    
    #predict the X of external set  
    arguments = [
        '--test_path', 'external_set_plus_'+feature+'_dmpnn_smile_active.csv',
        '--features_path','external_set_plus_'+feature+'_feature.csv',
        '--preds_path', 'dmpnn_'+feature+'_pred_external.csv',
        '--checkpoint_dir', 'dmpnn_'+feature+'_checkpoints'
    ]
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)
    file = pd.read_csv('dmpnn_'+feature+'_pred_external.csv')
    pred_df = pd.DataFrame(file)
    pred=pd.DataFrame()
    pred['pred'] =pred_df['active']
    df_external=pd.DataFrame()
    df_external['pred']=pred
    df_external.to_csv('pred_dmpnn_external_set_'+feature+'.csv',index=False)
    
    #calculate the AUC-ROC of the baseline models
    y_label = label
    y_pred = pred['pred']
    y_label = np.array(y_label)
    y_pred = np.array(y_pred)    
    fpr = dict()
    tpr = dict() 
    roc_auc = dict()
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    roc_auc = auc(fpr, tpr)
    name = 'dmpnn_'+feature
    name_all.append(name)
    globals()[name] = auc(fpr, tpr)
    auc_roc_test.append(globals()[name])

Command line
python /home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/hasee2/.local/share/jupyter/runtime/kernel-4ccb3dba-ed4a-4d91-9358-9f96db163a36.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': True,
 'data_path': 'main_set_plus_AtomPairFP_dmpnn_smile_active.csv',
 'data_weights_path': None,
 'dataset_

 36%|███████████████▎                           | 20/56 [00:00<00:00, 41.69it/s]Loss = 1.9569e-01, PNorm = 23.0947, GNorm = 1.1482, lr_0 = 7.7091e-04

 54%|███████████████████████                    | 30/56 [00:00<00:00, 42.54it/s]Loss = 2.2687e-01, PNorm = 23.1274, GNorm = 1.5813, lr_0 = 8.5273e-04

 71%|██████████████████████████████▋            | 40/56 [00:00<00:00, 43.28it/s]Loss = 1.9765e-01, PNorm = 23.1529, GNorm = 1.2020, lr_0 = 9.3455e-04

 89%|██████████████████████████████████████▍    | 50/56 [00:01<00:00, 42.75it/s]Loss = 2.1914e-01, PNorm = 23.1981, GNorm = 1.9274, lr_0 = 9.9701e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 41.95it/s]Loss = 2.6935e-01, PNorm = 23.2021, GNorm = 2.5491, lr_0 = 9.9552e-04

                                                                                
 86%|██████████████████████████████████████▌      | 6/7 [00:00<00:00, 50.13it/s]
                                                                                Val

 89%|██████████████████████████████████████▍    | 50/56 [00:01<00:00, 42.02it/s]Loss = 1.1202e-01, PNorm = 23.7998, GNorm = 0.8059, lr_0 = 7.1433e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 41.50it/s]
                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 43.59it/s]
                                                                                Validation auc = 0.925994
 20%|████████▊                                   | 6/30 [00:14<00:44,  1.87s/it]Epoch 6

  9%|███▉                                        | 5/56 [00:00<00:01, 44.27it/s]Loss = 6.0307e-02, PNorm = 23.8212, GNorm = 1.0182, lr_0 = 7.0372e-04

 27%|███████████▌                               | 15/56 [00:00<00:01, 40.02it/s]Loss = 4.8647e-02, PNorm = 23.8417, GNorm = 0.7849, lr_0 = 6.9328e-04

 45%|███████████████████▏                       | 25/56 [00:00<00:00, 41.58it/s]Loss = 7.9134e-02, PNorm = 23

  7%|███▏                                        | 4/56 [00:00<00:01, 39.29it/s]Loss = 3.5139e-02, PNorm = 24.1305, GNorm = 0.3784, lr_0 = 5.0419e-04

 23%|█████████▉                                 | 13/56 [00:00<00:01, 41.80it/s]Loss = 4.1332e-02, PNorm = 24.1470, GNorm = 0.3724, lr_0 = 4.9671e-04

 41%|█████████████████▋                         | 23/56 [00:00<00:00, 42.25it/s]Loss = 3.4081e-02, PNorm = 24.1554, GNorm = 0.2013, lr_0 = 4.8934e-04

 59%|█████████████████████████▎                 | 33/56 [00:00<00:00, 41.16it/s]Loss = 2.5718e-02, PNorm = 24.1691, GNorm = 0.1984, lr_0 = 4.8208e-04

 77%|█████████████████████████████████          | 43/56 [00:01<00:00, 42.19it/s]Loss = 4.7550e-02, PNorm = 24.1747, GNorm = 0.1782, lr_0 = 4.7492e-04

 95%|████████████████████████████████████████▋  | 53/56 [00:01<00:00, 41.49it/s]
                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 11.42it/s]
  

 71%|██████████████████████████████▋            | 40/56 [00:00<00:00, 40.30it/s]Loss = 3.6068e-02, PNorm = 24.3430, GNorm = 1.0697, lr_0 = 3.4077e-04

 89%|██████████████████████████████████████▍    | 50/56 [00:01<00:00, 40.43it/s]Loss = 3.3632e-02, PNorm = 24.3532, GNorm = 0.9872, lr_0 = 3.3572e-04

 98%|██████████████████████████████████████████▏| 55/56 [00:01<00:00, 41.18it/s]Loss = 1.3748e-01, PNorm = 24.3529, GNorm = 1.5043, lr_0 = 3.3522e-04

                                                                                
 71%|████████████████████████████████▏            | 5/7 [00:00<00:00, 46.95it/s]
                                                                                Validation auc = 0.929339
 50%|█████████████████████▌                     | 15/30 [00:28<00:23,  1.55s/it]Epoch 15

  9%|███▉                                        | 5/56 [00:00<00:01, 42.65it/s]Loss = 2.5938e-02, PNorm = 24.3555, GNorm = 0.1586, lr_0 = 3.3024e-04

 27%|███████████▌                     

...

Validating SMILES
Test size = 440


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.40it/s]

Saving predictions to dmpnn_Topology_pred_external.csv
Elapsed time = 0:00:01


In [6]:
#prepare the input of algorithm in layer 3
list_m=['xgb','fcnn','lr','dmpnn']
list_f=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column='main_value_all_'+m+feature,value=pred1)
        

external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column='external_value_all_'+m+feature,value=pred1)

pred_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_pred_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        pred_all_feature.insert(loc=len(pred_all_feature.columns),column='pred_value_all_'+m+feature,value=pred1)
        


In [7]:
#prepare the X and Y of main set
read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature

#prepare the X and Y of external set
read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature

#prepare the X of pred set
X_pred=pred_all_feature

#the algorithm in layer3
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])

#train algorithm
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)

#evaluate the model using the AUC-ROC
score = model.evaluate(X_e, y_e, verbose=0)
name_all.append('ensmble_model')
auc_roc_test.append(score[1])
print('AUC-ROC:'+str(score[1]))

#predict the Y of molecules in pred set
pred_smiles=model.predict(X_pred,
                  batch_size=None,
                  verbose="auto",
                  steps=None,
                  callbacks=None,
                  max_queue_size=10,
                  workers=1,
                  use_multiprocessing=False)
read_p=pd.read_csv('pred_set_plus_'+feature+'.csv')
df_p=pd.DataFrame(read_p)
X_p=df_p.iloc[:,0]
pred_mole=[]
pred_molecule=pd.DataFrame()
for i in pred_smiles:
    i=i[1]
    if i > 0.5:
        i='agonist'
    else:
        i='non-agonist'
    pred_mole.append(i)
pred_molecule['smiles']=X_p
pred_molecule['active']=pred_mole
pred_molecule.to_csv('predictive_results.csv',index=False)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.5487 - auc: 0.7322
Epoch 2/100
1/1 [==============================] - 0s 20ms/step - loss: 1.5664 - auc: 0.8068
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 1.3713 - auc: 0.8198
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 1.2335 - auc: 0.8206
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 1.1762 - auc: 0.8283
Epoch 6/100
1/1 [==============================] - 0s 20ms/step - loss: 1.0533 - auc: 0.8789
Epoch 7/100
1/1 [==============================] - 0s 22ms/step - loss: 0.9675 - auc: 0.9140
Epoch 8/100
1/1 [==============================] - 0s 19ms/step - loss: 0.9262 - auc: 0.9267
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 0.8633 - auc: 0.9437
Epoch 10/100
1/1 [==============================] - 0s 21ms/step - loss: 0.8129 - auc: 0.9537
Epoch 11/100
1/1 [==============================] - 0s 20ms/step - l

1/1 [==============================] - 0s 19ms/step - loss: 0.2382 - auc: 0.9902
Epoch 89/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2289 - auc: 0.9923
Epoch 90/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2364 - auc: 0.9899
Epoch 91/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2361 - auc: 0.9898
Epoch 92/100
1/1 [==============================] - 0s 18ms/step - loss: 0.2287 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2345 - auc: 0.9905
Epoch 94/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2337 - auc: 0.9910
Epoch 95/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2353 - auc: 0.9903
Epoch 96/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2241 - auc: 0.9925
Epoch 97/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2233 - auc: 0.9932
Epoch 98/100
1/1 [==============================] - 0s 19ms/step - loss: 

In [8]:
#predictive results
print(pred_molecule)

                                              smiles       active
0  OCCOc1ccc(c(c1)C)c1cccc(c1)CNc1ccc(c(c1)F)OCC(...  non-agonist
1   CC#CC(c1ccc(cc1)OCc1ccc(cc1)c1cccc(c1)OC)CC(=O)O      agonist
2  CC#C[C@H](c1ccc(cc1)OCc1ccc(cc1)c1cccc(c1)OC)C...      agonist
3  CC#C[C@@H](c1ccc(cc1)OCc1ccc(cc1)c1cccc(c1)OC)...      agonist
4            OCc1cc(CCC(=O)O)ccc1OCc1ccccc1SC(F)(F)F      agonist
5                  OC(=O)CSc1nnc(s1)SCc1cccc2c1cccc2      agonist
6     OC(=O)CS(=O)(=O)c1ccc(cc1)OCc1sc(nc1C)c1ccccc1  non-agonist
7  OC(=O)CS(=O)(=O)c1ccc(cc1)OCc1sc(nc1C)c1ccc(cc1)F  non-agonist
8  OC(=O)CS(=O)(=O)c1ccc(cc1)OCc1cccc(c1)c1c(C)cc...  non-agonist


In [9]:
#the results of evaluation based on external set
external_evaluate=pd.DataFrame()
external_evaluate['name']=name_all
external_evaluate['AUC-ROC']=auc_roc_test
external_evaluate.to_csv('results_eva.csv',index=False)
print(external_evaluate)

                name   AUC-ROC
0     xgb_AtomPairFP  0.917163
1         xgb_Charge  0.777441
2           xgb_MAP4  0.887630
3            xgb_MOE  0.853619
4       xgb_MorganFP  0.846325
5       xgb_Topology  0.769692
6    fcnn_AtomPairFP  0.924863
7        fcnn_Charge  0.905625
8          fcnn_MAP4  0.931906
9           fcnn_MOE  0.904917
10     fcnn_MorganFP  0.916023
11     fcnn_Topology  0.915181
12     lr_AtomPairFP  0.902690
13         lr_Charge  0.814334
14           lr_MAP4  0.849012
15            lr_MOE  0.837208
16       lr_MorganFP  0.797548
17       lr_Topology  0.765605
18  dmpnn_AtomPairFP  0.900866
19      dmpnn_Charge  0.778711
20        dmpnn_MAP4  0.833382
21         dmpnn_MOE  0.828905
22    dmpnn_MorganFP  0.809726
23    dmpnn_Topology  0.780046
24     ensmble_model  0.949695


In [10]:
#delete intermediate files
list_m=['xgb','fcnn','lr','rf','svm']
list_f=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        if os.path.exists('main_set_plus_'+feature+'_'+m+'_columns.csv'):
            os.remove('main_set_plus_'+feature+'_'+m+'_columns.csv')
        if os.path.exists('external_set_plus_'+feature+'_'+m+'_columns.csv'):
            os.remove('external_set_plus_'+feature+'_'+m+'_columns.csv')
        if os.path.exists(m+'_'+feature+'.csv'):
            os.remove(m+'_'+feature+'.csv')
        if os.path.exists('external_'+m+'_'+feature+'.csv'):
            os.remove('external_'+m+'_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__main_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__main_set_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__external_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__external_set_'+feature+'.csv')
        if os.path.exists('pred_'+m+'__pred_set_'+feature+'.csv'):
            os.remove('pred_'+m+'__pred_set_'+feature+'.csv')


In [11]:
#delete intermediate files
list_f=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
for feature in list_f:
    if os.path.exists('main_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('main_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('main_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('main_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_main.csv'):
        os.remove('dmpnn_'+feature+'_pred_main.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_pred.csv'):
        os.remove('dmpnn_'+feature+'_pred_pred.csv')
    if os.path.exists('external_set_plus_'+feature+'_dmpnn_smile_active.csv'):
        os.remove('external_set_plus_'+feature+'_dmpnn_smile_active.csv')
    if os.path.exists('external_set_plus_'+feature+'_dmpnn_feature.csv'):
        os.remove('external_set_plus_'+feature+'_dmpnn_feature.csv')
    if os.path.exists('dmpnn_'+feature+'_pred_external.csv'):
        os.remove('dmpnn_'+feature+'_pred_external.csv')
    if os.path.exists('pred_set_plus_'+feature+'_dmpnn_smile.csv'):
        os.remove('pred_set_plus_'+feature+'_dmpnn_smile.csv')
    if os.path.exists('pred_set_plus_'+feature+'_feature.csv'):
        os.remove('pred_set_plus_'+feature+'_feature.csv')
    if os.path.exists('external_set_plus_'+feature+'_feature.csv'):
        os.remove('external_set_plus_'+feature+'_feature.csv')